# Calculate power analysis given Linear Model effect sizes

In [1]:
library(pwr)
suppressPackageStartupMessages(library(dplyr))

In [2]:
output_file <- file.path("results", "power_analysis_cp_features_lm.tsv")
output_dp_file <- file.path("results", "power_analysis_dp_cyto_features_lm.tsv")

## Perform power analysis for CP features

In [3]:
# Load data
lm_results_file <- file.path("results", "linear_model_cp_features.tsv")
lm_results_df <- readr::read_tsv(
    lm_results_file,
    col_types = readr::cols(.default="d", feature="c")
)

print(dim(lm_results_df))
head(lm_results_df)

[1] 1043    5


feature,r2_score,cell_count_coef,Null_coef,WT_coef
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Cytoplasm_Number_Object_Number,0.309208943,0.033454088,-0.01216966,0.01216966
Cytoplasm_AreaShape_Area,0.316174998,-0.026660590,-0.17619770,0.17619770
Cytoplasm_AreaShape_BoundingBoxArea,0.167981769,-0.022168935,-0.05959405,0.05959405
Cytoplasm_AreaShape_BoundingBoxMaximum_X,0.008340445,0.002454425,-0.13958882,0.13958882
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,0.001684550,-0.003408135,0.04023927,-0.04023927
Cytoplasm_AreaShape_BoundingBoxMinimum_X,0.005920801,0.006485507,-0.08859939,0.08859939


In [4]:
# Load feature data (for calculating n)
data_dir <-file.path("..", "..", "..", "4_processing_features", "data")
cp_file <- file.path(data_dir, "nf1_sc_norm_cellprofiler.csv.gz")

cp_df <- readr::read_csv(
    cp_file,
    col_types = readr::cols(
        .default="d",
        Metadata_WellRow="c",
        Metadata_WellCol="c",
        Metadata_Well="c",
        Metadata_gene_name="c",
        Metadata_genotype="c"
    )
)

print(dim(cp_df))
head(cp_df, 3)

New names:
• `` -> `...1`


[1]  149 1056


...1,Metadata_WellRow,Metadata_WellCol,Metadata_number_of_singlecells,Metadata_gene_name,Metadata_genotype,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,⋯,Nuclei_Texture_SumVariance_RFP_3_02_256,Nuclei_Texture_SumVariance_RFP_3_03_256,Nuclei_Texture_Variance_GFP_3_00_256,Nuclei_Texture_Variance_GFP_3_01_256,Nuclei_Texture_Variance_GFP_3_02_256,Nuclei_Texture_Variance_GFP_3_03_256,Nuclei_Texture_Variance_RFP_3_00_256,Nuclei_Texture_Variance_RFP_3_01_256,Nuclei_Texture_Variance_RFP_3_02_256,Nuclei_Texture_Variance_RFP_3_03_256
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,C,6,12,NF1,WT,1,1,C6,1,⋯,3.1415403,3.2022729,-0.09735552,-0.096165089,-0.09420230,-0.10645635,3.3379688,3.3505284,3.2781681,3.3103705
1,C,6,12,NF1,WT,1,1,C6,2,⋯,0.3159240,0.2586328,-0.08797075,-0.069492845,-0.06553894,-0.09537677,0.3147762,0.3139198,0.3484196,0.3186928
2,C,6,12,NF1,WT,1,1,C6,3,⋯,0.2952335,0.3831610,0.06525064,0.005549586,-0.01521187,-0.02908654,0.3484921,0.3339402,0.3413119,0.3479994


In [5]:
# Define constants for power analysis
n_conditions <- 2  # NF1 WT and Null
n_samples <- dim(cp_df)[1]

u <- n_conditions - 1
v <- n_samples - u - 1
sig_level <- 0.05 / dim(lm_results_df)[1]
power <- 0.8

print(c(u, v))
print(sig_level)

[1]   1 147
[1] 4.793864e-05


In [6]:
# Given all R2 values perform power analysis
all_power_results <- list()
for (cp_feature in lm_results_df$feature) {
    # Subset to the given feature lm results
    lm_result_subset_df <- lm_results_df %>%
        dplyr::filter(feature == !!cp_feature)
    
    # Pull out the estimated R2 value
    r2_val <- lm_result_subset_df %>% dplyr::pull(r2_score)
    
    # The power estimate is undefined for r2_val = 1, skip if so
    if (r2_val == 1) {
        all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, NULL, NULL)
        next
    }
    
    # Transform R2 score to F2 effect size
    f2_val <- r2_val / (1 - r2_val)
    
    # Calculate power, note that v contains an estimate of sample size
    power_result <- pwr.f2.test(u = u, v = NULL, f2 = f2_val, sig.level = sig_level, power = power)
    
    # Calculate required sample size from the v formula
    estimated_sample_size <- power_result$v + u + 1
    
    # Save results for future visualization
    all_power_results[[cp_feature]] <- c(cp_feature, u, v, sig_level, power, estimated_sample_size)
    
}

In [7]:
power_results_df <- do.call(rbind, all_power_results) %>% dplyr::as_tibble()

colnames(power_results_df) <- c("feature", "u", "v", "sig_level", "power", "estimated_sample_size")

# Output to file
power_results_df %>%
    readr::write_tsv(output_file)

print(dim(power_results_df))
head(power_results_df)

Warning message in (function (..., deparse.level = 1) :
“number of columns of result is not a multiple of vector length (arg 65)”
Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.”


[1] 1043    6


feature,u,v,sig_level,power,estimated_sample_size
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cytoplasm_Number_Object_Number,1,147,4.79386385426654e-05,0.8,62.0461740901187
Cytoplasm_AreaShape_Area,1,147,4.79386385426654e-05,0.8,60.3294079142944
Cytoplasm_AreaShape_BoundingBoxArea,1,147,4.79386385426654e-05,0.8,127.52924923438
Cytoplasm_AreaShape_BoundingBoxMaximum_X,1,147,4.79386385426654e-05,0.8,2871.24963895602
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,1,147,4.79386385426654e-05,0.8,14278.4471735682
Cytoplasm_AreaShape_BoundingBoxMinimum_X,1,147,4.79386385426654e-05,0.8,4051.10195794731


## Perform power analysis for DP features

In [8]:
# Load data
lm_results_file <- file.path("results", "linear_model_dp_features.tsv")
lm_results_df <- readr::read_tsv(
    lm_results_file,
    col_types = readr::cols(.default="d", feature="c")
)

print(dim(lm_results_df))
head(lm_results_df)

[1] 3832    5


feature,r2_score,cell_count_coef,Null_coef,WT_coef
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
efficientnet_0,0.04655088,-1.313378e-02,0.098892191,-0.098892191
efficientnet_1,0.02633479,9.009381e-03,-0.246789424,0.246789424
efficientnet_2,0.32808305,-2.803389e-02,0.001363537,-0.001363537
efficientnet_3,0.02219074,5.271694e-03,-0.223836924,0.223836924
efficientnet_4,0.01098208,8.577969e-05,-0.116930972,0.116930972
efficientnet_5,0.02720802,8.035771e-03,-0.254575592,0.254575592


In [9]:
# Load feature data (for calculating n)
data_dir <-file.path("..", "..", "..", "4_processing_features", "data")
dp_file <- file.path(data_dir, "nf1_sc_norm_deepprofiler_cyto.csv.gz")

dp_df <- readr::read_csv(
    dp_file,
    col_types = readr::cols(
        .default="d",
        Metadata_Plate="c",
        Metadata_Well="c",
        Metadata_Site="c",
        Metadata_Plate_Map_Name="c",
        Metadata_DNA="c",
        Metadata_ER="c",
        Metadata_Actin="c",
        Metadata_Genotype="c",
        Metadata_Genotype_Replicate="c",
        Metadata_Model="c"
    )
)
print(dim(dp_df))
head(dp_df, 3)

[1]  256 3852


Location_Center_X,Location_Center_Y,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Plate_Map_Name,Metadata_DNA,Metadata_ER,Metadata_Actin,Metadata_Genotype,⋯,efficientnet_3830,efficientnet_3831,efficientnet_3832,efficientnet_3833,efficientnet_3834,efficientnet_3835,efficientnet_3836,efficientnet_3837,efficientnet_3838,efficientnet_3839
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
650.4225,736.7067,1,D6,3,1_D6_3,../../../../1_preprocessing_data/Corrected_Images/D6_01_1_3_DAPI_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/D6_01_2_3_GFP_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/D6_01_3_3_RFP_001_IllumCorrect.tif,WT,⋯,-0.03401407,0.8595177,-0.02571327,-0.7295710,0.2672313,-0.8473657,-1.09008160,0.5667855,-0.02436295,0.2304633
949.1827,256.7347,1,F6,2,1_F6_2,../../../../1_preprocessing_data/Corrected_Images/F6_01_1_2_DAPI_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_2_2_GFP_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_3_2_RFP_001_IllumCorrect.tif,WT,⋯,-0.02110868,0.1855114,-0.61222690,2.9258058,-0.9277611,-0.3043499,-0.03106468,0.6265455,-0.40805978,-1.2116158
454.4601,265.7970,1,F6,2,1_F6_2,../../../../1_preprocessing_data/Corrected_Images/F6_01_1_2_DAPI_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_2_2_GFP_001_IllumCorrect.tif,../../../../1_preprocessing_data/Corrected_Images/F6_01_3_2_RFP_001_IllumCorrect.tif,WT,⋯,-0.15838182,-0.8102113,-0.72087693,0.1278688,1.1994534,1.9774389,-0.84440530,-0.2991374,-0.77516640,-0.2921428


In [10]:
# Define constants for power analysis (n_samples is different from CP features)
n_conditions <- 2  # NF1 WT and Null
n_samples <- dim(dp_df)[1]

u <- n_conditions - 1
v <- n_samples - u - 1
sig_level <- 0.05 / dim(lm_results_df)[1]
power <- 0.8

print(c(u, v))
print(sig_level)

[1]   1 254
[1] 1.304802e-05


In [11]:
# Given all R2 values perform power analysis
all_power_results <- list()
for (dp_feature in lm_results_df$feature) {
    # Subset to the given feature lm results
    lm_result_subset_df <- lm_results_df %>%
        dplyr::filter(feature == !!dp_feature)
    
    # Pull out the estimated R2 value
    r2_val <- lm_result_subset_df %>% dplyr::pull(r2_score)
    
    # The power estimate is undefined for r2_val = 1, skip if so
    if (r2_val == 1) {
        all_power_results[[dp_feature]] <- c(dp_feature, u, v, sig_level, NULL, NULL)
        next
    }
    
    # Transform R2 score to F2 effect size
    f2_val <- r2_val / (1 - r2_val)
    
    # Calculate power, note that v contains an estimate of sample size
    power_result <- pwr.f2.test(u = u, v = NULL, f2 = f2_val, sig.level = sig_level, power = power)
    
    # Calculate required sample size from the v formula
    estimated_sample_size <- power_result$v + u + 1
    
    # Save results for future visualization
    all_power_results[[dp_feature]] <- c(dp_feature, u, v, sig_level, power, estimated_sample_size)
    
}

In [12]:
power_results_df <- do.call(rbind, all_power_results) %>% dplyr::as_tibble()

colnames(power_results_df) <- c("feature", "u", "v", "sig_level", "power", "estimated_sample_size")

# Output to file
power_results_df %>%
    readr::write_tsv(output_dp_file)

print(dim(power_results_df))
head(power_results_df)

[1] 3832    6


feature,u,v,sig_level,power,estimated_sample_size
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
efficientnet_0,1,254,1.30480167014614e-05,0.8,563.521954130131
efficientnet_1,1,254,1.30480167014614e-05,0.8,1009.59107700252
efficientnet_2,1,254,1.30480167014614e-05,0.8,64.8259187122834
efficientnet_3,1,254,1.30480167014614e-05,0.8,1201.40694229486
efficientnet_4,1,254,1.30480167014614e-05,0.8,2445.51410341455
efficientnet_5,1,254,1.30480167014614e-05,0.8,976.625087249966
